# Spark SQL vs. PySpark DataFrame API

Compare the Spark SQL interface with the PySpark DataFrame API so you can choose the right tool for each task.


## Prerequisites

- Spark environment capable of running PySpark notebooks.
- Shared dataset `notebooks/data/orders_demo.csv` available.
- Familiarity with basic SQL and the PySpark DataFrame API.


## Load DataFrame and Create a View

We load the shared orders dataset once, then create a temporary view for SQL queries. This mirrors typical production jobs that support both Python and SQL entry points.


In [ ]:
from pathlib import Path
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName('SparkSQLvsPySpark').getOrCreate()

repo_root = Path.cwd()
if (repo_root / 'notebooks').exists():
    data_path = repo_root / 'notebooks' / 'data' / 'orders_demo.csv'
else:
    data_path = Path('..') / 'data' / 'orders_demo.csv'

orders_df = (
    spark.read
    .option('header', True)
    .option('inferSchema', True)
    .csv(str(data_path))
    .withColumn('order_date', F.to_date('order_date'))
)

orders_df.createOrReplaceTempView('orders')
orders_df.orderBy('order_date', 'region').show()


## Selecting Columns and Filtering Rows

Both APIs can express the same logic. DataFrame code stays in Python, while Spark SQL keeps the query in SQL syntax.


In [ ]:
# PySpark DataFrame API
filtered_df = (
    orders_df
      .select('order_date', 'region', 'orders')
      .where((F.col('region') == 'north') & (F.col('orders') >= 12))
      .orderBy('order_date')
)
filtered_df.show()


In [ ]:
# Spark SQL equivalent
spark.sql(
    '''
    SELECT order_date, region, orders
    FROM orders
    WHERE region = "north" AND orders >= 12
    ORDER BY order_date
    '''
).show()


## Adding Derived Columns

Conditionals and built-in functions are available in both interfaces. PySpark uses Python functions under `pyspark.sql.functions`; SQL uses expressions and CASE statements.


In [ ]:
# PySpark DataFrame API
pyspark_enriched = orders_df.withColumn(
    'demand_level',
    F.when(F.col('orders') >= 14, 'high').otherwise('steady')
)
pyspark_enriched.orderBy('order_date', 'region').show()


In [ ]:
# Spark SQL equivalent
spark.sql(
    '''
    SELECT *,
           CASE WHEN orders >= 14 THEN 'high' ELSE 'steady' END AS demand_level
    FROM orders
    ORDER BY order_date, region
    '''
).show()


## Aggregations and Grouping

Aggregations feel similar, but DataFrame calls chain methods, while SQL exposes the familiar GROUP BY clause.


In [ ]:
# PySpark DataFrame API
pyspark_summary = (
    orders_df
      .groupBy('region')
      .agg(
          F.sum('orders').alias('total_orders'),
          F.avg('orders').alias('avg_orders'),
      )
      .orderBy('region')
)
pyspark_summary.show()


In [ ]:
# Spark SQL equivalent
spark.sql(
    '''
    SELECT region,
           SUM(orders) AS total_orders,
           AVG(orders) AS avg_orders
    FROM orders
    GROUP BY region
    ORDER BY region
    '''
).show()


## When to Use Each Interface

- **PySpark DataFrame API** keeps logic in Python, making it easy to integrate with functions, tests, or dynamic query construction.
- **Spark SQL** shines when analysts contribute SQL snippets or when you register views consumed by BI tools.
- Mix both: build reusable DataFrame transformations, then expose them as views for downstream SQL consumers.


## Clean Up

Stop the SparkSession when you finish experimentation.


In [ ]:
spark.stop()


## Exercises

- Recreate a DataFrame transformation of your choice using both the DataFrame API and Spark SQL, verifying the results match.
- Add a parameterized filter that accepts a region name from Python and injects it safely into a SQL query.
- Compare the physical plans (`explain(mode='formatted')`) for the DataFrame and SQL versions of the same query—note any differences.
